In [1]:
import pandas as pd

In [2]:
cbb = pd.read_csv('../Final_Project_DE/train_data.csv')
cbb24 = pd.read_csv('../Final_Project_DE/test_data.csv')

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from pygam import LinearGAM, s, f

# Filter rows with non-null SEED values and convert SEED to integers
cbb_model = cbb[cbb['SEED'].notnull()]
cbb_model['SEED'] = cbb_model['SEED'].astype(int)

# Define feature columns and the target variable
features = [
    'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR', 'TORD', 'ORB',
    'DRB', 'FTR', 'FTRD', 'ADJ_T', 'WAB'
]
X = cbb_model[features]
y = cbb_model['SEED']

# Extract the features from cbb24 for prediction
cbb24_seedless = cbb24[features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
cbb24_scaled = scaler.transform(cbb24_seedless)

# Fit a GAM model
gam = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + 
                s(7) + s(8) + s(9) + s(10) + s(11) + s(12)).fit(X_train, y_train)

# Predict on the test set and compute MAE
gam.gridsearch(X_train, y_train, progress=True)
y_pred = gam.predict(X_test)
y_pred_rounded = np.round(y_pred).clip(1, 16)  # Clip predictions to valid seed range
mae = mean_absolute_error(y_test, y_pred_rounded)
print(f"Mean Absolute Error (MAE): {mae}")

# Predict seeds for cbb24
cbb24_predictions = gam.predict(cbb24_scaled)
cbb24['predicted_seed_score'] = cbb24_predictions  # Store raw predictions for ranking

# Rank teams based on predicted seed scores
cbb24['rank'] = cbb24['predicted_seed_score'].rank(method='min', ascending=True)

# Assign seeds to top 68 teams
cbb24['predicted_seed_with_update'] = np.nan  # Initialize with NaN
top_68 = cbb24.nsmallest(68, 'predicted_seed_score')  # Select top 68 teams

# Create seed values for 68 teams
seed_values = []
for seed in range(1, 17):
    if seed == 11 or seed == 16:
        seed_values.extend([seed] * 6)  # 6 teams for seeds 11 and 16
    else:
        seed_values.extend([seed] * 4)  # 4 teams for other seeds

# Assign seeds to the top 68 teams
top_68['predicted_seed_with_update'] = seed_values

# Update the main DataFrame with the seeded teams
cbb24.update(top_68) 
cbb24 = cbb24.sort_values(by='predicted_seed_with_update')


/var/folders/mc/2wjfdchj6vsffbrpfbfgqw4w0000gn/T/ipykernel_86529/172987666.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbb_model['SEED'] = cbb_model['SEED'].astype(int)
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:05
 18% (2 of 11) |####                     | Elapsed Time: 0:00:01 ETA:   0:00:05
 27% (3 of 11) |######                   | Elapsed Time: 0:00:01 ETA:   0:00:04
 36% (4 of 11) |#########                | Elapsed Time: 0:00:02 ETA:   0:00:03
 45% (5 of 11) |###########              | Elapsed Time: 0:00:02 ETA:   0:00:03
 54% (6 of 11) |#############            | Elapsed Time: 0:00:03 ETA:   0:00:02
 63% (7 of 11) |###

Mean Absolute Error (MAE): 0.9338235294117647


In [4]:
cbb24['delta_predicted_seed_update'] = cbb24['SEED'] - cbb24['predicted_seed_with_update']

In [5]:
cbb24[['TEAM', 'SEED', 'delta_predicted_seed_update']].sort_values('SEED').dropna(subset=['delta_predicted_seed_update']).head(20)

,TEAM,SEED,delta_predicted_seed_update
0,Houston,1.0,0.0
1,Connecticut,1.0,0.0
2,Purdue,1.0,0.0
8,North Carolina,1.0,-1.0
3,Iowa St.,2.0,1.0
7,Marquette,2.0,0.0
5,Arizona,2.0,-1.0
6,Tennessee,2.0,-1.0
11,Illinois,3.0,1.0
24,Kentucky,3.0,-2.0


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.barttorvik.com/"
response = requests.get(url)

# Parse the HTML
soup = BeautifulSoup(response.text, "html.parser")

# Inspect the page structure and find the table or relevant data
# Example: Extracting table data
table = soup.find("table")
rows = table.find_all("tr")

data = []
for row in rows:
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
df = df.dropna(how="all").reset_index(drop=True)

df.columns = ['RK', 'TEAM', 'CONF', 'G', 'REC', 'ADJOE', 'ADJDE', 'BARTHAG', "EFG_O", "EFG_D", 'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', "2P_O", "2P_D", "3P_O", "3P_D", '3PR', '3PRD', 'ADJ_T', 'WAB']
df = df[[
    "TEAM", "CONF", "G", "ADJOE", "ADJDE", "BARTHAG", 
    "EFG_O", "EFG_D", "TOR", "TORD", "ORB", "DRB", "FTR", "FTRD", 
    "2P_O", "2P_D", "3P_O", "3P_D", "ADJ_T", "WAB"
]]
df.to_csv("current_cbb_live_data.csv", index=False)


In [7]:
current_data = pd.read_csv('../Final_Project_DE/current_cbb_live_data.csv')

In [8]:
# predict seeds:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from pygam import LinearGAM, s, f

# Filter rows with non-null SEED values and convert SEED to integers
cbb_model = cbb[cbb['SEED'].notnull()]
cbb_model['SEED'] = cbb_model['SEED'].astype(int)

# Define feature columns and the target variable
features = [
    'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR', 'TORD', 'ORB',
    'DRB', 'FTR', 'FTRD', 'ADJ_T', 'WAB'
]
X = cbb_model[features]
y = cbb_model['SEED']

# Extract the features from cbb24 for prediction
current_data_seedless = current_data[features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
current_data_scaled = scaler.transform(current_data_seedless)

# Fit a GAM model
gam = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + 
                s(7) + s(8) + s(9) + s(10) + s(11) + s(12)).fit(X_train, y_train)

# Predict on the test set and compute MAE
gam.gridsearch(X_train, y_train, progress=True)
y_pred = gam.predict(X_test)
y_pred_rounded = np.round(y_pred).clip(1, 16)  # Clip predictions to valid seed range
mae = mean_absolute_error(y_test, y_pred_rounded)
print(f"Mean Absolute Error (MAE): {mae}")

# Predict seeds for cbb24
current_data_predictions = gam.predict(current_data_scaled)
current_data['predicted_seed_score'] = current_data_predictions  # Store raw predictions for ranking

# Rank teams based on predicted seed scores
current_data['rank'] = current_data['predicted_seed_score'].rank(method='min', ascending=True)

# Assign seeds to top 68 teams
current_data['predicted_seed_with_update'] = np.nan  # Initialize with NaN
top_68 = current_data.nsmallest(68, 'predicted_seed_score')  # Select top 68 teams

# Create seed values for 68 teams
seed_values = []
for seed in range(1, 17):
    if seed == 11 or seed == 16:
        seed_values.extend([seed] * 6)  # 6 teams for seeds 11 and 16
    else:
        seed_values.extend([seed] * 4)  # 4 teams for other seeds

# Assign seeds to the top 68 teams
top_68['predicted_seed_with_update'] = seed_values

# Update the main DataFrame with the seeded teams
current_data.update(top_68) 
current_data = current_data.sort_values(by='predicted_seed_with_update')


/var/folders/mc/2wjfdchj6vsffbrpfbfgqw4w0000gn/T/ipykernel_86529/1139329377.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbb_model['SEED'] = cbb_model['SEED'].astype(int)
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:06
 18% (2 of 11) |####                     | Elapsed Time: 0:00:01 ETA:   0:00:05
 27% (3 of 11) |######                   | Elapsed Time: 0:00:01 ETA:   0:00:04
 36% (4 of 11) |#########                | Elapsed Time: 0:00:02 ETA:   0:00:04
 45% (5 of 11) |###########              | Elapsed Time: 0:00:03 ETA:   0:00:03
 54% (6 of 11) |#############            | Elapsed Time: 0:00:03 ETA:   0:00:03
 63% (7 of 11) |#

Mean Absolute Error (MAE): 0.9338235294117647


In [9]:
from datetime import datetime
current_date = datetime.now().strftime("%Y%m%d")
file_name = f"seeding_2025_{current_date}.csv"


In [10]:
current_data

,TEAM,CONF,G,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,predicted_seed_score,rank,predicted_seed_with_update
3,Auburn,SEC,4,119.6300,93.5600,0.944940,64.0200,39.8900,14.1480,17.4188,...,24.5430,70.6100,39.1110,37.6670,27.8530,70.5103,1.1700,8.292364,3.0,1.0
4,Tennessee vs. 11 Baylor,SEC,5,114.8270,90.0400,0.942350,60.6100,40.8170,18.5207,21.5620,...,27.0680,60.8230,39.9140,40.2280,27.9570,69.1197,1.2600,8.475900,4.0,1.0
5,Kansas,B12,5,118.8400,93.6800,0.939260,57.3370,44.2480,12.8200,17.5185,...,33.2169,60.2290,46.1830,34.3138,27.3410,71.1750,1.3400,8.242315,2.0,1.0
7,Alabama,SEC,5,119.7200,95.4240,0.931280,56.2480,45.8760,13.9440,13.6322,...,33.3172,64.9400,45.6780,30.8232,30.6123,73.6800,0.9900,8.135396,1.0,1.0
17,Marquette,BE,5,113.7380,94.0110,0.899418,52.6126,47.6108,11.9800,24.7150,...,30.2114,58.9410,48.8128,30.7235,30.6123,70.6970,1.5100,8.828410,7.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Mercyhurst,NEC,6,93.8356,112.1348,0.114236,46.2262,52.7232,16.9140,23.1350,...,40.1272,44.8292,54.9254,32.5180,32.4167,68.3259,-1.3227,13.128523,239.0,NaN
358,Prairie View A&M,SWAC,5,94.7348,113.3359,0.113236,41.6340,54.1263,14.5590,17.1202,...,45.5314,41.1333,56.1278,28.3287,33.7205,73.4100,-2.8335,13.098499,235.0,NaN
360,Arkansas Pine Bluff,SWAC,6,94.1353,115.6364,0.085836,49.6197,60.7359,23.6346,13.0338,...,33.2169,52.8145,67.2364,29.5259,35.6262,72.5310,-2.7334,12.851821,211.0,NaN
361,Coppin St.,MEAC,7,89.3362,110.8329,0.077436,38.5355,59.9355,20.7286,17.3192,...,39.5260,39.9340,61.3343,23.6344,38.7321,68.9215,-4.0361,13.273490,252.0,NaN


In [11]:
current_data['TEAM'] = current_data['TEAM'].str.split('vs.').str[0].str.strip()

In [12]:
current_data

,TEAM,CONF,G,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,predicted_seed_score,rank,predicted_seed_with_update
3,Auburn,SEC,4,119.6300,93.5600,0.944940,64.0200,39.8900,14.1480,17.4188,...,24.5430,70.6100,39.1110,37.6670,27.8530,70.5103,1.1700,8.292364,3.0,1.0
4,Tennessee,SEC,5,114.8270,90.0400,0.942350,60.6100,40.8170,18.5207,21.5620,...,27.0680,60.8230,39.9140,40.2280,27.9570,69.1197,1.2600,8.475900,4.0,1.0
5,Kansas,B12,5,118.8400,93.6800,0.939260,57.3370,44.2480,12.8200,17.5185,...,33.2169,60.2290,46.1830,34.3138,27.3410,71.1750,1.3400,8.242315,2.0,1.0
7,Alabama,SEC,5,119.7200,95.4240,0.931280,56.2480,45.8760,13.9440,13.6322,...,33.3172,64.9400,45.6780,30.8232,30.6123,73.6800,0.9900,8.135396,1.0,1.0
17,Marquette,BE,5,113.7380,94.0110,0.899418,52.6126,47.6108,11.9800,24.7150,...,30.2114,58.9410,48.8128,30.7235,30.6123,70.6970,1.5100,8.828410,7.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Mercyhurst,NEC,6,93.8356,112.1348,0.114236,46.2262,52.7232,16.9140,23.1350,...,40.1272,44.8292,54.9254,32.5180,32.4167,68.3259,-1.3227,13.128523,239.0,NaN
358,Prairie View A&M,SWAC,5,94.7348,113.3359,0.113236,41.6340,54.1263,14.5590,17.1202,...,45.5314,41.1333,56.1278,28.3287,33.7205,73.4100,-2.8335,13.098499,235.0,NaN
360,Arkansas Pine Bluff,SWAC,6,94.1353,115.6364,0.085836,49.6197,60.7359,23.6346,13.0338,...,33.2169,52.8145,67.2364,29.5259,35.6262,72.5310,-2.7334,12.851821,211.0,NaN
361,Coppin St.,MEAC,7,89.3362,110.8329,0.077436,38.5355,59.9355,20.7286,17.3192,...,39.5260,39.9340,61.3343,23.6344,38.7321,68.9215,-4.0361,13.273490,252.0,NaN


In [13]:
current_data[['TEAM', 'predicted_seed_with_update']].sort_values('predicted_seed_with_update').dropna(subset=['predicted_seed_with_update']).to_csv(file_name)

In [14]:
current_data

,TEAM,CONF,G,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,predicted_seed_score,rank,predicted_seed_with_update
3,Auburn,SEC,4,119.6300,93.5600,0.944940,64.0200,39.8900,14.1480,17.4188,...,24.5430,70.6100,39.1110,37.6670,27.8530,70.5103,1.1700,8.292364,3.0,1.0
4,Tennessee,SEC,5,114.8270,90.0400,0.942350,60.6100,40.8170,18.5207,21.5620,...,27.0680,60.8230,39.9140,40.2280,27.9570,69.1197,1.2600,8.475900,4.0,1.0
5,Kansas,B12,5,118.8400,93.6800,0.939260,57.3370,44.2480,12.8200,17.5185,...,33.2169,60.2290,46.1830,34.3138,27.3410,71.1750,1.3400,8.242315,2.0,1.0
7,Alabama,SEC,5,119.7200,95.4240,0.931280,56.2480,45.8760,13.9440,13.6322,...,33.3172,64.9400,45.6780,30.8232,30.6123,73.6800,0.9900,8.135396,1.0,1.0
17,Marquette,BE,5,113.7380,94.0110,0.899418,52.6126,47.6108,11.9800,24.7150,...,30.2114,58.9410,48.8128,30.7235,30.6123,70.6970,1.5100,8.828410,7.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Mercyhurst,NEC,6,93.8356,112.1348,0.114236,46.2262,52.7232,16.9140,23.1350,...,40.1272,44.8292,54.9254,32.5180,32.4167,68.3259,-1.3227,13.128523,239.0,NaN
358,Prairie View A&M,SWAC,5,94.7348,113.3359,0.113236,41.6340,54.1263,14.5590,17.1202,...,45.5314,41.1333,56.1278,28.3287,33.7205,73.4100,-2.8335,13.098499,235.0,NaN
360,Arkansas Pine Bluff,SWAC,6,94.1353,115.6364,0.085836,49.6197,60.7359,23.6346,13.0338,...,33.2169,52.8145,67.2364,29.5259,35.6262,72.5310,-2.7334,12.851821,211.0,NaN
361,Coppin St.,MEAC,7,89.3362,110.8329,0.077436,38.5355,59.9355,20.7286,17.3192,...,39.5260,39.9340,61.3343,23.6344,38.7321,68.9215,-4.0361,13.273490,252.0,NaN


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report
from pygam import LogisticGAM, s

# Preprocessing POSTSEASON column
cbb = cbb[cbb['POSTSEASON'].notnull()]  # Filter rows with POSTSEASON values
label_encoder = LabelEncoder()
cbb['POSTSEASON_LABEL'] = label_encoder.fit_transform(cbb['POSTSEASON'])  # Encode POSTSEASON

# Select features for modeling
postseason_features = [
    'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR', 'TORD', 'ORB',
    'DRB', 'FTR', 'FTRD', 'ADJ_T', 'WAB'
]

# Define X and y for training
X_postseason = cbb[postseason_features]
y_postseason = cbb['POSTSEASON_LABEL']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_postseason, y_postseason, test_size=0.2, random_state=42
)

# One-hot encode the target variable for OvR
encoder = OneHotEncoder(sparse_output=False)
y_train_onehot = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test_onehot = encoder.transform(y_test.values.reshape(-1, 1))

# Train one LogisticGAM per class
gam_models = []
for class_idx in range(y_train_onehot.shape[1]):
    gam = LogisticGAM(
        s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) +
        s(7) + s(8) + s(9) + s(10) + s(11) + s(12)
    ).fit(X_train, y_train_onehot[:, class_idx])
    gam.gridsearch(X_train, y_train, progress=True)
    gam_models.append(gam)

# Predict probabilities for each class on test data
probabilities = np.column_stack([gam.predict_proba(X_test) for gam in gam_models])

# Assign the class with the highest probability
y_pred = np.argmax(probabilities, axis=1)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Apply the model to cbb24 data
cbb24_seeded = current_data.dropna(subset=['predicted_seed_with_update'])

X_cbb24 = cbb24_seeded[postseason_features]
postseason_probs = np.column_stack([gam.predict_proba(X_cbb24) for gam in gam_models])

# Define the number of teams per label
label_counts = {
    'Champion': 1,
    'Runner-Up': 1,
    'Final Four': 2,
    'Elite Eight': 4,
    'Sweet Sixteen': 8,
    'Round of 32': 16,
    'Round of 64': 32
}

# Assign labels systematically
num_labels = len(label_counts)
assigned_labels = np.full(len(cbb24_seeded), num_labels - 1)  # Default to lowest label
remaining_teams = list(range(len(cbb24_seeded)))

# Loop through each label, starting with the highest rank (Champion)
for label_idx, (label, count) in enumerate(label_counts.items()):
    if len(remaining_teams) == 0:
        break
    label_probs = postseason_probs[remaining_teams, label_idx]
    top_indices = np.argsort(label_probs)[-count:]  # Get indices of top `count` teams
    top_team_indices = [remaining_teams[i] for i in top_indices]  # Map back to original indices
    for i in top_team_indices:
        assigned_labels[i] = label_idx
    remaining_teams = [team for team in remaining_teams if team not in top_team_indices]  # Exclude assigned teams

# Map assigned labels back to POSTSEASON names
cbb24_seeded['predicted_postseason_label'] = assigned_labels

postseason_mapping = {
    0: 'Winner',
    1: 'Runner-Up',
    2: 'Final Four',
    3: 'Elite Eight',
    4: 'Sweet Sixteen',
    5: 'Round of 32',
    6: 'Round of 64'
}

# Map the numerical labels to their descriptions
cbb24_seeded['predicted_postseason_description'] = cbb24_seeded['predicted_postseason_label'].map(postseason_mapping)

# Display the predictions
cbb24_seeded

ValueError: y data is not in domain of logit link function. Expected domain: [0.0, 1.0], but found [0.0, 7.0]

In [ ]:
cbb24_seeded.sort_values('predicted_seed_with_update').head(20)

,TEAM,CONF,G,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,...,2P_D,3P_O,3P_D,ADJ_T,WAB,predicted_seed_score,rank,predicted_seed_with_update,predicted_postseason_label,predicted_postseason_description
3,Auburn,SEC,4,119.630,93.560,0.944940,64.0200,39.8900,14.1480,17.4188,...,39.1110,37.6670,27.8530,70.5103,1.1700,8.292364,3.0,1.0,6,Round of 64
4,Tennessee,SEC,5,114.827,90.040,0.942350,60.6100,40.8170,18.5207,21.5620,...,39.9140,40.2280,27.9570,69.1197,1.2600,8.475900,4.0,1.0,6,Round of 64
5,Kansas,B12,5,118.840,93.680,0.939260,57.3370,44.2480,12.8200,17.5185,...,46.1830,34.3138,27.3410,71.1750,1.3400,8.242315,2.0,1.0,6,Round of 64
7,Alabama,SEC,5,119.720,95.424,0.931280,56.2480,45.8760,13.9440,13.6322,...,45.6780,30.8232,30.6123,73.6800,0.9900,8.135396,1.0,1.0,6,Round of 64
17,Marquette,BE,5,113.738,94.011,0.899418,52.6126,47.6108,11.9800,24.7150,...,48.8128,30.7235,30.6123,70.6970,1.5100,8.828410,7.0,2.0,6,Round of 64
19,Purdue,B10,5,116.712,96.945,0.894420,59.0190,49.0149,16.6126,13.4327,...,52.1198,43.8900,29.5980,67.9285,0.7210,8.653958,5.0,2.0,6,Round of 64
10,Baylor,B12,5,118.650,96.839,0.912611,55.3630,56.2312,16.8135,22.8410,...,53.5223,39.1390,40.2341,68.2262,1.2500,8.825597,6.0,2.0,4,Sweet Sixteen
9,Kentucky (H) 330 Jackson St.,SEC,4,116.911,95.018,0.916110,59.1180,43.0360,13.7420,13.0338,...,48.8128,41.8150,23.0900,72.5300,1.1800,9.053051,8.0,2.0,2,Final Four
2,Gonzaga,WCC,5,122.210,94.714,0.949530,57.0420,42.6280,11.2200,20.7760,...,48.0116,36.3910,22.5600,70.4106,1.5200,9.071775,9.0,3.0,6,Round of 64
0,Houston (H) 147 Hofstra,B12,3,118.370,89.610,0.960610,55.7550,44.1460,14.4560,27.8200,...,49.4144,48.2100,25.4220,65.5354,-0.1384,9.476405,12.0,3.0,1,Runner-Up
